In [22]:
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Function to save data to CSV file
def save_to_csv(data):
    headers = ["Category", "Title", "Classification", "Date", "URL", "Content", "Correction"]
    with open('MISBAR_FACTCHECK_13-04-3pm.csv', 'a', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        for row in data:
            writer.writerow(row)

# Initialize the WebDriver
driver = webdriver.Chrome()

try:
    # Open the webpage
    driver.get('https://misbar.com/factcheck')

    # Wait until the page finishes loading
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'li.dropdown-item')))

    # Find all the category elements
    category_elements = driver.find_elements(By.CSS_SELECTOR, 'li.dropdown-item')
    print(category_elements)

    # Iterate through each category
    for category_element in category_elements:
        print("Processing the : ",category_element ,"Category element")
        # Get the category name
        #category_name = category_element.text
        
        # Click on the category element
        category_element.click()
        
        # Wait until the page finishes loading
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.factcheck-feed-landing')))

        # Initialize counter for tracking containers
        container_counter = 0
        
        # Initialize set to keep track of URLs already scraped
        scraped_urls = set()
                # Loop to click "Load more" button 10 times
        for _ in range(40): #for more data make it bigger
            # Find the "Load more" button
            load_more_button = driver.find_element(By.CSS_SELECTOR, 'button.load-more')
            # If "Load more" button is found, click on it
            if load_more_button:
                load_more_button.click()
                # Give some time for the additional articles to load
                time.sleep(2)
            else:
                # If "Load more" button is not found, break the loop
                break
        
        # Loop to handle "Show more" button
        #while True:
            # Find and extract the news articles for the current category
            news_articles = driver.find_elements(By.CSS_SELECTOR, 'div.factcheck-item')
            
            # Output the articles for the current category
            for article in news_articles:
                url = article.find_element(By.CSS_SELECTOR, 'div.img-large-verticle-landing > a').get_attribute('href')

                # Skip if URL has already been scraped
                if url in scraped_urls:
                    continue
                
                #driver.switch_to.window(driver.window_handles[-1])
                
                title = article.find_element(By.CSS_SELECTOR, 'h3.card-title').text
                date = article.find_element(By.CSS_SELECTOR, 'span.article--card_date').text 
                try:
                    classification = article.find_element(By.CSS_SELECTOR, 'div.classification > span').text
                    print(classification)
                    category_name = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.category-type'))).text
                    print(category_name)
                except Exception as e:
                    content = "Category not available"
                # Open the article in a new tab
                driver.execute_script("window.open('{}', '_blank');".format(url))
                # Switch to the newly opened tab
                driver.switch_to.window(driver.window_handles[-1])
                #try:
                    
                  
                #except Exception as e:
                    #print("Error extracting category name:", e)
                # Find the content of the article
                try:
                    correction_content = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.deep-dive--article_analysis')))
                    
                    content_element = WebDriverWait(driver, 10).until(
                        EC.presence_of_element_located((By.CSS_SELECTOR, 'div.deep-dive--article_story'))
                    )
                    # Extract the text content from <p> elements
                    paragraphs = content_element.find_elements(By.TAG_NAME, 'p')
                    content = ' '.join(paragraph.text for paragraph in paragraphs)
                    #print("content \n")
                    #print(content)

                    corr = correction_content.find_elements(By.TAG_NAME, 'p')
                    correction = ' '.join(cor.text for cor in corr)
                    #print("correction \n")
                    #print(correction)
                    #print(" \n")
                except Exception as e:
                    content = "Content or Correction not available"

                # Close the tab
                driver.close()
                # Switch back to the main tab
                driver.switch_to.window(driver.window_handles[0])

                # Append the data to the CSV file
                save_to_csv([[category_name, title, classification, date, url, content , correction]])


except Exception as e:
    print("An error occurred:", e)

finally:
    # Close the WebDriver
    driver.quit()


[<selenium.webdriver.remote.webelement.WebElement (session="209466f620a1414bf40171415a30ef39", element="f.BF5F49B0D25467D3B92AF0362F67285D.d.E5169EE57DAB5EE2A98745EE79E9E9FD.e.36")>, <selenium.webdriver.remote.webelement.WebElement (session="209466f620a1414bf40171415a30ef39", element="f.BF5F49B0D25467D3B92AF0362F67285D.d.E5169EE57DAB5EE2A98745EE79E9E9FD.e.37")>, <selenium.webdriver.remote.webelement.WebElement (session="209466f620a1414bf40171415a30ef39", element="f.BF5F49B0D25467D3B92AF0362F67285D.d.E5169EE57DAB5EE2A98745EE79E9E9FD.e.68")>, <selenium.webdriver.remote.webelement.WebElement (session="209466f620a1414bf40171415a30ef39", element="f.BF5F49B0D25467D3B92AF0362F67285D.d.E5169EE57DAB5EE2A98745EE79E9E9FD.e.69")>, <selenium.webdriver.remote.webelement.WebElement (session="209466f620a1414bf40171415a30ef39", element="f.BF5F49B0D25467D3B92AF0362F67285D.d.E5169EE57DAB5EE2A98745EE79E9E9FD.e.70")>, <selenium.webdriver.remote.webelement.WebElement (session="209466f620a1414bf40171415a30ef